In [50]:
! pip install tensorflow
! pip install sklearn

ERROR: Could not find a version that satisfies the requirement scikitlearn (from versions: none)
ERROR: No matching distribution found for scikitlearn


In [52]:
! pip install sklearn

     |████████████████████████████████| 22.2 MB 39.2 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 114.8 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=69a830ff0431c2f9ad21124316ac12048afce59ff89c8a0677134daae6577359
  Stored in directory: /home/dnanexus/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [26]:
# download model input data from DNANexus folder
#%%bash
!dx download "model_input" -r

[===========================================================>] Completed 2,291 of 2,291 bytes (100%) /opt/notebooks/model_input/README.mdd
[===========================================================>] Completed 638 of 638 bytes (100%) /opt/notebooks/model_input/pheno_data.tsvv
[===========================================================>] Completed 317,021 of 317,021 bytes (100%) /opt/notebooks/model_input/SV_VCFs.INS.gene_scores_matrix.tsv.gzz
[===========================================================>] Completed 293,991 of 293,991 bytes (100%) /opt/notebooks/model_input/SV_VCFs.DEL.gene_scores_matrix.tsv.gzz
[===========================================================>] Completed 628 of 628 bytes (100%) /opt/notebooks/model_input/SV_VCFs.INV.gene_scores_matrix.tsv.gzz
[===========================================================>] Completed 610 of 610 bytes (100%) /opt/notebooks/model_input/SV_VCFs.DUP-TANDEM.gene_scores_matrix.tsv.gzz
[=============================================

In [53]:
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from os import listdir
import pickle
from sklearn.preprocessing import StandardScaler

In [39]:
%clear

# PARAMETERS

In [54]:
# analysis parameters
survival_th = 2 #threshold betw. low and high survival
# PATHS and files
input_path = 'model_input'
output_path = 'model_output'

ignore_files = ['README.md'] # files that are not feature input data
label_file = 'pheno_data.tsv' # name of the label file

# model_parameters
L1_REGULARIZATION = 0.5 #l1 regularization parameter
EPOCHS = 100    #number of training epochs
VALIDATION_SPLIT = 0.1    #fraction of data held out for validation
BATCH_SIZE = 1    # size of training batches


# Load and preprocess data

In [55]:
def fill_nans(df):
    #frist substitute nans with coulumns mean, that does not affect performance
    filled_df = df.fillna(df.mean())
    #for all nan columns, put zeros as constant value
    filled_df = filled_df.fillna(0)
    return filled_df


In [56]:
#load all feature files and clean them
file_names = [f for f in listdir(input_path)]
dataframes = {}
for fname in file_names:
    if not (fname in ignore_files or fname==label_file):
        df = pd.read_csv(input_path+'/'+fname,sep='\t',index_col=0)
        df = fill_nans(df)
        dataframes[fname] = df

modalities = dataframes.keys()
# load phenotipes for label construction
labels_df = pd.read_csv(input_path+'/'+label_file,sep='\t',index_col=0)

# to add checks that all IDS are in the same order in all dataframes

In [57]:
# prepares input matrices
X0 = [x.values for x in dataframes.values()]
# z-score inputs in each modality
scaler = StandardScaler()
X = [scaler.fit_transform(x) for x in X0]
# prepare labels for bynary classification, low survival=1, high survival=0 
Y = np.array([1 if y<survival_th else 0 for y in labels_df['Phe3_Surv_years'].values])

# MODEL DEFINITION

In [58]:
input_layers = []
for m,m_name in enumerate(modalities):
    input_layer = keras.Input(shape=X[m].shape[1:], name=f"m{m+1}_input")
    input_layers.append(input_layer)

m_output_layers = []
for m,m_name in enumerate(modalities):
    output_layer = layers.Dense(1,activation='relu',
                                kernel_regularizer=regularizers.l1(l1=L1_REGULARIZATION),
                                name=f"m{m+1}_output")(input_layers[m])
    m_output_layers.append(output_layer)
# concatenation layer
concatenation = layers.concatenate(m_output_layers,name='concatenated_modalitites')
# logisitc regression layer
prediction = layers.Dense(1, name="output_layer")(concatenation)

model = keras.Model(
    inputs=input_layers,
    outputs=[prediction],
)

In [59]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
m1_input (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
m2_input (InputLayer)           [(None, 2349)]       0                                            
__________________________________________________________________________________________________
m3_input (InputLayer)           [(None, 2177)]       0                                            
__________________________________________________________________________________________________
m4_input (InputLayer)           [(None, 34)]         0                                            
____________________________________________________________________________________________

# MODEL COMPILATION

In [60]:
# model compilation
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])

# MODEL TRAINING

In [61]:
# train model
history = model.fit(X,Y,epochs=EPOCHS,validation_split=VALIDATION_SPLIT, batch_size=BATCH_SIZE, shuffle=True,verbose=2)

Epoch 1/100
18/18 - 1s - loss: 194.7795 - accuracy: 0.5556 - val_loss: 103.5495 - val_accuracy: 0.0000e+00
Epoch 2/100
18/18 - 0s - loss: 75.7724 - accuracy: 0.5556 - val_loss: 73.8611 - val_accuracy: 0.0000e+00
Epoch 3/100
18/18 - 0s - loss: 43.1094 - accuracy: 0.4444 - val_loss: 32.3451 - val_accuracy: 0.0000e+00
Epoch 4/100
18/18 - 0s - loss: 29.5418 - accuracy: 0.6111 - val_loss: 28.3232 - val_accuracy: 0.0000e+00
Epoch 5/100
18/18 - 0s - loss: 44.4831 - accuracy: 0.6111 - val_loss: 41.0271 - val_accuracy: 1.0000
Epoch 6/100
18/18 - 0s - loss: 41.7812 - accuracy: 0.7222 - val_loss: 31.0159 - val_accuracy: 1.0000
Epoch 7/100
18/18 - 0s - loss: 29.5417 - accuracy: 0.6667 - val_loss: 21.9514 - val_accuracy: 1.0000
Epoch 8/100
18/18 - 0s - loss: 22.5118 - accuracy: 0.6111 - val_loss: 21.7708 - val_accuracy: 0.0000e+00
Epoch 9/100
18/18 - 0s - loss: 40.6814 - accuracy: 0.6111 - val_loss: 55.2251 - val_accuracy: 1.0000
Epoch 10/100
18/18 - 0s - loss: 47.8604 - accuracy: 0.6111 - val_loss

# Feature ranking

In [66]:
#feature rakings
output_dfs = {}

mod_w = model.get_layer('output_layer').get_weights()[0].flatten()
for m,m_name in enumerate(modalities):
    features_w = model.get_layer(f'm{m+1}_output').get_weights()[0].flatten()
    tot_w = features_w*mod_w[m]
    mod_order = tot_w.argsort()[::-1] # to rank from larger to smaller
    mod_ranks = mod_order.argsort()
    
    df = pd.DataFrame(columns=['Feature_ID','Rank','Score'])
    df['Feature_ID'] = dataframes[m_name].columns
    df['Rank'] = mod_ranks
    df['Score'] = tot_w
    
    output_dfs[m_name] = df.sort_values(by=['Rank'])
    
# modality weights
mod_df = pd.DataFrame(columns=['Modality','Weight'])
mod_df['Modality'] = modalities
mod_df['Weight'] = mod_w
    

# SAVE OUTPUT

In [71]:
# make model_output dir
Path(output_path).mkdir(parents=True, exist_ok=True)

# save_dataframes
for key in output_dfs.keys():
    output_dfs[key].to_csv(output_path +'/'+key,sep='\t',index=False,header=output_dfs[key].columns)

# save modality weigths
mod_df.to_csv(output_path +'/modality_weights',sep='\t',index=False,header=mod_df.columns)

# save trained model
model_path = output_path+'/'+'model'
Path(model_path).mkdir(parents=True, exist_ok=True)
model.save(model_path)

# save training history
with open(model_path+'/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


INFO:tensorflow:Assets written to: model_output/model/assets


In [48]:
# upload output to DNAnexus
!dx upload -r model_output

[===========================================================>] Uploaded 1,247 of 1,247 bytes (100%) model_output/HERV_K_Insertions.txt
[===========================================================>] Uploaded 23,990 of 23,990 bytes (100%) model_output/SV_VCFs.INS.gene_scores_matrix.tsv.gz
[===========================================================>] Uploaded 24,203 of 24,203 bytes (100%) model_output/SV_VCFs.DEL.gene_scores_matrix.tsv.gz
[===========================================================>] Uploaded 435 of 435 bytes (100%) model_output/SV_VCFs.INV.gene_scores_matrix.tsv.gz
[===========================================================>] Uploaded 418 of 418 bytes (100%) model_output/SV_VCFs.DUP-TANDEM.gene_scores_matrix.tsv.gz
[===========================================================>] Uploaded 444,624 of 444,624 bytes (100%) model_output/SNP_VCFs.genotype_matrix.tsv.gz
[===========================================================>] Uploaded 589,517 of 589,517 bytes (100%) model